In [62]:
import random
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [63]:
PATH = './input/'
df = pd.read_csv(PATH + "webtoon_total_final.csv")

In [64]:
df = df[['title','score', 'genre']]
df.genre = df.genre.str.strip('['']')

In [65]:
tmp = pd.get_dummies(df.genre)

In [66]:
def col_change(df):
    for col in df.columns:
        if ',' in col:
            col_sep = col.split(', ')
            df[col_sep[0]] = df[col_sep[0]] + df[col]
            df[col_sep[1]] = df[col_sep[1]] + df[col]
            df.drop(columns=[f'{col}'], inplace=True)
    return df

In [67]:
col_change(tmp)
tmp

,'드라마','로맨스','무협','스릴러','액션','일상','코믹','판타지','학원'
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
3228,0,1,0,0,0,0,0,0,0
3229,0,0,0,1,0,0,0,0,0
3230,0,1,0,0,0,0,0,0,0
3231,0,0,0,0,0,0,0,1,1


In [68]:
genre_df = col_change(pd.get_dummies(df.genre))
df = pd.concat([df.title, df.score, genre_df], axis=1)
df

,title,score,'드라마','로맨스','무협','스릴러','액션','일상','코믹','판타지','학원'
0,참교육,9.9510,0,0,0,0,1,0,0,0,0
1,신의 탑,7.3420,0,0,0,0,0,0,0,1,0
2,뷰티풀 군바리,9.7290,1,0,0,0,0,0,0,0,0
3,윈드브레이커,9.5430,0,0,0,0,0,0,0,0,1
4,퀘스트지상주의,9.8890,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3228,진심이 닿다,9.7825,0,1,0,0,0,0,0,0,0
3229,드림사이드,9.8300,0,0,0,1,0,0,0,0,0
3230,김 비서가 왜 그럴까,9.9200,0,1,0,0,0,0,0,0,0
3231,더스크 하울러,9.9350,0,0,0,0,0,0,0,1,1


In [69]:
df.columns = df.columns.str.strip('\'')
df

,title,score,드라마,로맨스,무협,스릴러,액션,일상,코믹,판타지,학원
0,참교육,9.9510,0,0,0,0,1,0,0,0,0
1,신의 탑,7.3420,0,0,0,0,0,0,0,1,0
2,뷰티풀 군바리,9.7290,1,0,0,0,0,0,0,0,0
3,윈드브레이커,9.5430,0,0,0,0,0,0,0,0,1
4,퀘스트지상주의,9.8890,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3228,진심이 닿다,9.7825,0,1,0,0,0,0,0,0,0
3229,드림사이드,9.8300,0,0,0,1,0,0,0,0,0
3230,김 비서가 왜 그럴까,9.9200,0,1,0,0,0,0,0,0,0
3231,더스크 하울러,9.9350,0,0,0,0,0,0,0,1,1


In [70]:
def one_genre(title):
    """
    장르 하나의 
    """
    g_row = df[df.title == title]
    genres = g_row[df.columns[2:]]
    return genres.values

def genres(title_list):
    genre_list = [0]*9
    for title in title_list:
        genre_list = genre_list + one_genre(title)
    return genre_list


## 시청 목록과 장르 유사도 높은 웹툰 중 평점 높은 10개

In [106]:
score = genre_df.to_numpy()

def genre_model(title_list):
    local_score = np.append(score, genres(title_list), axis=0)
    cosine_similar = cosine_similarity(local_score, local_score)
    cosine_similar_data = pd.DataFrame(cosine_similar)

    genre_user = cosine_similar_data.tail(1).T.sort_values(by=cosine_similar_data.columns[-1], ascending=False)
    max_score = genre_user.values[1][0]
    
    max_index = list(genre_user[(genre_user.values) == max_score].index)

    if df.shape[0] in max_index:
        max_index.remove(df.shape[0])

    sorted_df = df.loc[max_index].sort_values(by='score', ascending=False)

    if len(max_index) < 10:
        return sorted_df
    
    else:
        min_score = sorted_df.iloc[9].score
        ind = 0
        for i in range(10):
            if sorted_df.iloc[i].score == min_score:
                ind = i
                break
        randnum = sorted_df[sorted_df.score == min_score].shape[0]
        randlist = random.sample(range(ind, ind + randnum), 10 - ind)
        tdf1, tdf2 = sorted_df[:ind], sorted_df.iloc[randlist]
        sorted_df = pd.concat([tdf1, tdf2])

        return sorted_df

In [125]:
g = genre_model(['바니와 오빠들', '신락도', '신의 탑', '오무라이스 잼잼'])
g

,title,score,드라마,로맨스,무협,스릴러,액션,일상,코믹,판타지,학원
548,숲속의 담,9.990000,0,0,0,0,0,0,0,1,0
435,메트로헌터,9.987000,0,0,0,0,0,0,0,1,0
93,천마는 평범하게 살 수 없다,9.987000,0,0,0,0,0,0,0,1,0
6,호랑신랑뎐,9.987000,0,0,0,0,0,0,0,1,0
901,전자오락수호대,9.986000,0,0,0,0,0,0,0,1,0
675,살아남은 로맨스,9.984000,0,0,0,0,0,0,0,1,0
1042,가담항설,9.983000,0,0,0,0,0,0,0,1,0
326,갓 오브 하이스쿨,9.982000,0,0,0,0,0,0,0,1,0
595,합격시켜주세용,9.982000,0,0,0,0,0,0,0,1,0
804,롤랑롤랑,9.981111,0,0,0,0,0,0,0,1,0


In [105]:
r = range(3,12)
r = random.sample(r,3)
print(r)

[9, 6, 8]


In [47]:
l = [0,1,4]
g.iloc[l]

,title,score,드라마,로맨스,무협,스릴러,액션,일상,코믹,판타지,학원
2229,창백한 말,9.990,0,1,0,0,0,0,0,1,0
1931,샬롯에게는 다섯 명의 제자가 있다,9.987,0,1,0,0,0,0,0,1,0
2026,슬프게도 이게 내 인생,9.984,0,0,0,0,0,1,1,0,0


In [ ]:
g.loc[2226]['무협']

0